# 1.	Python

a)В python2.7 есть ф-ции range, и xrange, в чем разница? 

b)	Написать на python 2 или 3 генератор чисел фибоначи.

In [22]:
def fibonacci_generator(n):
    df=np.zeros(n+1, dtype=int) 
    df[0]=0
    df[1]=1
    for i in range(2, n+1):
        df[i]=df[i-1]+df[i-2]
    yield df

In [24]:
# вывести первые 10 числе фибоначи
n = 10
for num in fibonacci_generator(n):
              print(num)

[ 0  1  1  2  3  5  8 13 21 34 55]


# 2.	Написать Sql-запросы

1.	Вывести отдел с наибольшим числом сотрудников;

2.	Вывести список сотрудников, получающих заработную плату выше, чем у руководителя;

# 3.	Machine Learning

In [149]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
from pandas import ExcelWriter
from sklearn.grid_search import GridSearchCV
from sklearn.cross_validation import KFold
from sklearn.cross_validation import cross_val_score
from sklearn.cross_validation import train_test_split
from sklearn.metrics import f1_score, coverage_error, label_ranking_loss, mean_squared_error
#classificators
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier

In [150]:
scaler = StandardScaler()
label = LabelEncoder()

In [151]:
df=pd.read_excel('train.xlsx')
test=pd.read_excel('test.xlsx')

In [152]:
df2=df.copy()
test2=test.copy()

Были созданы фичи: к-во переводов с региона, к-во переводов на регион, максимальное, минимальное, среднее количество, сумма, std переводов от 1 компании к другой и т.д.

In [153]:
df.head()

,Node,Feat4,Y,Column0,Column4,Column6,Column7,Column10,Column12,Column13,...,wCOUNT_of_NodeLeft,wCOUNT_DISTINCT_of_NodeLeft,wMEAN_of_Feat1,wMAX_of_Feat1,wSTD_of_Feat1,wMEAN_of_Feat2,wMAX_of_Feat2,wSTD_of_Feat2,wCOUNT_DISTINCT_of_reg1,wCOUNT_DISTINCT_of_reg2
0,0,0,35,20.0,1.0,4.0,19.0,2.0,6.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,0,7,20.0,1.0,4.0,19.0,2.0,6.0,1.0,...,4.0,4.0,0.047986,0.139597,0.065871,0.005768,0.013179,0.005402,2.0,1.0
2,2,0,35,20.0,1.0,4.0,19.0,2.0,6.0,1.0,...,4.0,1.0,0.008725,0.008725,0.000000,0.000524,0.000524,0.000000,1.0,1.0
3,3,0,42,20.0,1.0,4.0,19.0,2.0,6.0,1.0,...,1.0,1.0,0.000000,0.000000,NaN,0.001064,0.001064,NaN,1.0,1.0
4,4,0,37,20.0,1.0,4.0,19.0,2.0,6.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [154]:
df.isnull().values.any()

True

In [210]:
def preproc(data):
    data=data.fillna(0)
    X=scaler.fit_transform(data)
    return X

In [211]:
df.Y.value_counts()[:15]

34    15441
35    14190
32     6466
50     6375
37     3878
45     3816
40     2806
52     2792
0      2551
46     2510
36     2245
33     2080
55     1788
7      1466
58     1294
Name: Y, dtype: int64

In [212]:
Y=df.Y
df=df.drop('Y',axis=1)
X=preproc(df)

In [215]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=0)

In [243]:
clf = RandomForestClassifier(random_state=0,class_weight="balanced")
tuned_parameters = [{'min_samples_split':np.arange(1, 4, 1)},
                     {'n_estimators':np.arange(5, 25, 5)},
                      {'max_depth':np.arange(1, 6, 1)}]
gs = GridSearchCV(clf, tuned_parameters, cv=8,scoring="f1_weighted" ,verbose=True)

In [244]:
gs.fit(X_train, y_train)

Fitting 8 folds for each of 12 candidates, totalling 96 fits


[Parallel(n_jobs=1)]: Done  49 tasks       | elapsed:  3.4min
[Parallel(n_jobs=1)]: Done  96 out of  96 | elapsed:  4.8min finished


GridSearchCV(cv=8, error_score='raise',
       estimator=RandomForestClassifier(bootstrap=True, class_weight='balanced',
            criterion='gini', max_depth=None, max_features='auto',
            max_leaf_nodes=None, min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=0, verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid=[{'min_samples_split': array([1, 2, 3])}, {'n_estimators': array([ 5, 10, 15, 20])}, {'max_depth': array([1, 2, 3, 4, 5])}],
       pre_dispatch='2*n_jobs', refit=True, scoring='f1_weighted',
       verbose=True)

In [245]:
gs.best_estimator_

RandomForestClassifier(bootstrap=True, class_weight='balanced',
            criterion='gini', max_depth=None, max_features='auto',
            max_leaf_nodes=None, min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=20, n_jobs=1,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

In [246]:
preds_train=pd.Series(gs.predict(X_train))
preds_train.value_counts()[:15]

34    10397
35     9454
32     4345
50     4277
37     2633
45     2550
40     1879
52     1834
0      1724
46     1674
36     1477
33     1385
55     1222
7      1025
58      834
dtype: int64

In [247]:
print ("MSE: ", mean_squared_error(y_train, preds_train))
print ("F-мера: ", f1_score(y_train, preds_train))

MSE:  1.83412765957
F-мера:  0.993046201687


Смотрим на валидацию

In [248]:
print ("MSE: ",mean_squared_error(y_test, gs.predict(X_test)))
print ("F-мера: ",f1_score(y_test, gs.predict(X_test)))

MSE:  224.21716142
F-мера:  0.179487877491


# TEST

In [217]:
test=preproc(test)

In [249]:
ans=gs.predict(test)
ans.value_counts()[:15]

34    10025
35     2896
50     1334
32     1199
7       648
0       523
40      489
46      376
57      261
55      260
33      255
27      177
37      158
15      145
45      139
dtype: int64

In [237]:
ans=pd.concat([test2["Node"],ans], axis=1)
ans.to_csv("answer.csv")

,Node,0
0,8,7
1,13,21
2,15,50
3,18,7
4,19,59
5,22,7
6,24,21
7,26,18
8,27,35
9,30,59
